In [ ]:
"""
Read the following data set:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/
Rename the columns as per the description from this file:
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
Task:
Create a sql db from adult dataset and name it sqladb
1. Select 10 records from the adult sqladb
2. Show me the average hours per week of all men who are working in private sector
3. Show me the frequency table for education, occupation and relationship, separately
4. Are there any people who are married, working in private sector and having a masters
degree
5. What is the average, minimum and maximum age group for people working in
different sectors
6. Calculate age distribution by country
7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and
'capital-loss'

"""

In [31]:
import csv
import sqlite3

In [32]:
import pandas as pd

In [33]:
#Provide Sqlite3 client connection
db = sqlite3.connect("sqladb.db")
cursor = db.cursor()

In [34]:
#Create Table with the naming convention
# - is replaced with _ in column names since sqlite doesn't allow - in column names
try :
    db.execute('DROP table adultTable')
    print("Dropped table adultTable")
except:
    a=1
finally:
    db.execute("CREATE TABLE adultTable (age INT(2),workclass VARCHAR(255), fnlwgt INT(5), education VARCHAR(255), education_num INT(5), marital_status VARCHAR(255), occupation VARCHAR(255), relationship VARCHAR(255), race VARCHAR(255), sex VARCHAR(255), capital_gain INT(20), capital_loss INT(20), hours_per_week INT(20), native_country VARCHAR(255), moreThan50K VARCHAR(255) )")
    print("Created table adultTable")

Created table adultTable


In [35]:

#Load data from sample.csv to table
with open ('sample.csv','r') as csvfile:
    table_data = csv.reader(csvfile)
    for row in table_data:
            try:
                listrow = row[0].split(',')
                db.execute('INSERT INTO adultTable VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',listrow)
            except:
                break        
# Commit is important to reflect the changes
db.commit()

In [36]:
#1. Select 10 records from the adult sqladb
query = "SELECT * FROM adultTable LIMIT 10"
pd.read_sql(query, db)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,moreThan50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [37]:
# 2) Average hours per week for all man who are working in private sector
query = "SELECT AVG(hours_per_week) average_hours_per_week FROM adultTable WHERE trim(workclass) = 'Private' and trim(sex) = 'Male' "
pd.read_sql(query, db)

,average_hours_per_week
0,42.221226


In [38]:
#3. Show me the frequency table for education, occupation and relationship, separately
#    a) education
query = "SELECT education , COUNT(education) frequency FROM adultTable GROUP BY education "
pd.read_sql(query, db)


,education,frequency
0,10th,933
1,11th,1175
2,12th,433
3,1st-4th,168
4,5th-6th,333
5,7th-8th,646
6,9th,514
7,Assoc-acdm,1067
8,Assoc-voc,1382
9,Bachelors,5355


In [39]:
#    b) occupation
query = "SELECT occupation, COUNT(occupation) frequency FROM adultTable GROUP BY occupation "
pd.read_sql(query, db)

,occupation,frequency
0,?,1843
1,Adm-clerical,3770
2,Armed-Forces,9
3,Craft-repair,4099
4,Exec-managerial,4066
5,Farming-fishing,994
6,Handlers-cleaners,1370
7,Machine-op-inspct,2002
8,Other-service,3295
9,Priv-house-serv,149


In [40]:
#    a) relationship
query = "SELECT relationship, COUNT(relationship) frequency FROM adultTable GROUP BY relationship "
pd.read_sql(query, db)

,relationship,frequency
0,Husband,13193
1,Not-in-family,8305
2,Other-relative,981
3,Own-child,5068
4,Unmarried,3446
5,Wife,1568


In [41]:
# 4) Are there any people who are married, working in private sector and having a masters degree 
query = " SELECT * FROM adultTable WHERE TRIM(UPPER(marital_status)) LIKE ('MARRIED%') and trim(workclass) = 'Private' and trim(education) = 'Masters'"

pd.read_sql(query, db)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,moreThan50K
0,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
1,33,Private,202051,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
2,76,Private,124191,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
3,31,Private,99928,Masters,14,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,50,United-States,<=50K
4,35,Private,138992,Masters,14,Married-civ-spouse,Prof-specialty,Other-relative,White,Male,7298,0,40,United-States,>50K
5,34,Private,142897,Masters,14,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,Male,7298,0,35,Taiwan,>50K
6,62,Private,270092,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,>50K
7,41,Private,445382,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1977,65,United-States,>50K
8,33,Private,208405,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,>50K
9,49,Private,192776,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1977,45,United-States,>50K


In [42]:
# 5) Average, minimum and maximum age group for people working in different sectors
query = " SELECT workclass as sector, AVG(age) averageAge, MIN(age) minAge, MAX(age) maxAge  FROM adultTable GROUP BY workclass "
pd.read_sql(query, db)

,sector,averageAge,minAge,maxAge
0,?,40.960240,17,90
1,Federal-gov,42.590625,17,90
2,Local-gov,41.751075,17,90
3,Never-worked,20.571429,17,30
4,Private,36.797585,17,90
5,Self-emp-inc,46.017025,17,84
6,Self-emp-not-inc,44.969697,17,90
7,State-gov,39.436055,17,81
8,Without-pay,47.785714,19,72


In [43]:
#6. Calculate age distribution by country
query = "SELECT Native_Country, Age, COUNT(Age) 'Count' FROM adultTable GROUP BY Native_Country, Age "
pd.read_sql(query, db)

,native_country,age,Count
0,?,17,2
1,?,18,8
2,?,19,5
3,?,20,10
4,?,21,11
5,?,22,12
6,?,23,6
7,?,24,14
8,?,25,11
9,?,26,18


In [44]:
#7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'
query = " SELECT capital_gain, capital_loss , ( capital_gain - capital_loss ) 'Net_Capital_Gain' FROM adulttable "
pd.read_sql(query, db)

,capital_gain,capital_loss,Net_Capital_Gain
0,2174,0,2174
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
5,0,0,0
6,0,0,0
7,0,0,0
8,14084,0,14084
9,5178,0,5178
